In [1]:
from transformers import pipeline
import os
os.environ['TRANSFORMERS_CACHE'] = '/data2/hanyings/.cache'
unmasker = pipeline('fill-mask', model='bert-base-uncased')
#words = unmasker("Hello I'm a [MASK] boy.")
from nltk.corpus import wordnet as wn
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "zh"
def find_tokenidx(token, sentence_lst):
  for i in range(len(sentence_lst)):
    if token in sentence_lst[i]:
      return i
def mutant_pairs(examples):
  pairs=[]
  inputs = [ex["translation"][source_lang] for ex in examples]
  for i, input in enumerate(inputs):
    input_list = input.split()
    tokens = examples[i]['top_tokens']
    # sys = wn.synsets(input_list[tokens])
    # idx = 0
    # while 1:
    #   if len(wn.synset(sys[idx].name()).lemmas()) >= 2:
    #     break
    #   idx += 1
    # candidate = [str(lemma.name()) for lemma in wn.synset(sys[idx].name()).lemmas()]
    print(tokens)
    token_idx = find_tokenidx(tokens[0], input_list)
    print(input_list[token_idx])
    input_list[token_idx] = "[MASK]"
    candidate = unmasker(" ".join(input_list))
    #print(candidate)
    mutants = []
    for c in candidate[:3]:
      mutant = input_list.copy()
      mutant[token_idx] = c["token_str"]
      mutants.append(" ".join(mutant))
    pairs.append({'en': input, 'mutants': mutants})
  return pairs

/data2/hanyings/conda_envs/tnmt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-05 13:36:39.791748: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 13:36:45.154517: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:$LD_LIBRARY_PATH
2023-03-05 13:36:45.155208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

In [2]:
mutant_pairs(result)

NameError: name 'result' is not defined

In [3]:
from nltk.corpus import wordnet as wn

sys = wn.synsets("comfortable", pos = 'a')

#[str(lemma.name()) for lemma in wn.synset(sys[0].name()).lemmas()]
sys[0].similar_tos()[0].lemma_names()

['cozy', 'cosy', 'snug']

In [4]:
from nltk.corpus import wordnet as wn

sys = wn.synsets("Europe")

#[str(lemma.name()) for lemma in wn.synset(sys[0].name()).lemmas()]

print(sys[0].part_holonyms())
# sys[0].pos()
# sys[0].wup_similarity(sys[0].hyponyms()[0])

[Synset('eurasia.n.01'), Synset('west.n.01')]


In [5]:
# functions in synset
[func for func in dir(wn.synsets("run")[0]) if func[0] != "_"]    

['acyclic_tree',
 'also_sees',
 'attributes',
 'causes',
 'closure',
 'common_hypernyms',
 'definition',
 'entailments',
 'examples',
 'frame_ids',
 'hypernym_distances',
 'hypernym_paths',
 'hypernyms',
 'hyponyms',
 'in_region_domains',
 'in_topic_domains',
 'in_usage_domains',
 'instance_hypernyms',
 'instance_hyponyms',
 'jcn_similarity',
 'lch_similarity',
 'lemma_names',
 'lemmas',
 'lexname',
 'lin_similarity',
 'lowest_common_hypernyms',
 'max_depth',
 'member_holonyms',
 'member_meronyms',
 'min_depth',
 'mst',
 'name',
 'offset',
 'part_holonyms',
 'part_meronyms',
 'path_similarity',
 'pos',
 'region_domains',
 'res_similarity',
 'root_hypernyms',
 'shortest_path_distance',
 'similar_tos',
 'substance_holonyms',
 'substance_meronyms',
 'topic_domains',
 'tree',
 'usage_domains',
 'verb_groups',
 'wup_similarity']

In [6]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet as wn

def tag(sentence):
 words = word_tokenize(sentence)
 words = pos_tag(words)
 return words

def paraphraseable(tag):
 return tag.startswith('N') or tag.startswith('V') or tag.startswith('J') or tag.startswith('R')

def pos(tag):
 if tag.startswith('N'):
  return wn.NOUN
 elif tag.startswith('V'):
  return wn.VERB
 elif tag.startswith('J'):
  return wn.ADJ
 elif pos.startswith("R"):
        return wn.ADV

def synonyms(word, tag):
  word_synset = wn.synsets(word, pos(tag))[0]
  def check_same(lst):
    return not all(x.lemma_names()[0] == word for x in lst)
  if tag.startswith('J'):
    lemma_lists = [ss for ss in word_synset.similar_tos()]

  else:
    if word_synset.hyponyms():
      lemma_lists = word_synset.hyponyms()
    elif word_synset.hypernyms():
      lemma_lists = word_synset.hypernyms()
    elif word_synset.part_holonyms():
      lemma_lists = word_synset.part_holonyms()
    else:
      for func in dir(word_synset):
        if func[0] != "_":
          item = getattr(word_synset, func)
          try:
            temp_lemma_lst = item()
          except Exception as e: 
            print(e)
            continue
          if isinstance(temp_lemma_lst, list) and temp_lemma_lst !=[]  and not isinstance(temp_lemma_lst[0], list)  and check_same(temp_lemma_lst):
            lemma_lists = temp_lemma_lst
            break
  
  # for lemma in lemma_lists:
  #   if pos_tag(lemma) == tag:
  #     print(lemma.lemma_names())
  lemmas =[]
  for lemma in lemma_lists:
    sim = lemma.wup_similarity(wn.synsets(word, pos=pos(tag))[0])
    if sim >= 0.5:
      lemmas.append((lemma.lemma_names()[0], sim))
    #print(lemma.lemma_names())
  #lemmas = [(lemma.lemma_names(), lemma.wup_similarity(wn.synsets(word)[0])) for lemma in lemma_lists ]
  return lemmas

def synonymIfExists(sentence, index):
   tags = tag(sentence)
   word, t = tags[index]
   if paraphraseable(t):
    syns = synonyms(word, t)
    if syns:
      syns_list = [s for s in syns if s[0].lower() != word.lower()]
      return word, syns_list
    return word, []

def paraphrase(sentence):
 return [x for x in synonymIfExists(sentence)]
get=[]
get=synonymIfExists("Europe is being cautious in the name of avoiding debt and defending the euro", 8)
print("synonyms",get)

synonyms ('avoiding', [])


In [ ]:
wn.synset('fabian.s.02').lch_similarity(wn.synsets('cautious', pos="a")[0])
wn.synsets('cautious', pos="a")[0]

WordNetError: Computing the lch similarity requires Synset('fabian.s.02') and Synset('cautious.a.01') to have the same part of speech.

In [ ]:
from transformers import pipeline
import os
os.environ['TRANSFORMERS_CACHE'] = '/data2/hanyings/.cache'
unmasker = pipeline('fill-mask', model='bert-base-uncased')
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine
# Load pre-trained model tokenizer (vocabulary)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

from nltk.corpus import wordnet as wn
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "zh"
def find_tokenidx(token, sentence_lst):
  for i in range(len(sentence_lst)):
    if token.lower() in sentence_lst[i].lower():
      return i
def token_vec_sum(text, word):
  marked_text = "[CLS] " + text + " [SEP]"
  tokenized_text = bert_tokenizer.tokenize(marked_text)
  index = find_tokenidx(word, tokenized_text)
  print(index)
  print(tokenized_text)
  indexed_tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1] * len(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  model = BertModel.from_pretrained('bert-base-uncased',
                                    output_hidden_states = True, # Whether the model returns all hidden-states.
                                    )
  model.eval()
  with torch.no_grad():
      outputs = model(tokens_tensor, segments_tensors)
      hidden_states = outputs[2]
      token_embeddings = torch.stack(hidden_states, dim=0)
      token_embeddings = torch.squeeze(token_embeddings, dim=1)
      token_embeddings = token_embeddings.permute(1,0,2)
      token_vecs_sum = []
      for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)
  return token_vecs_sum[index]
def similarity(origin, mutant, word, mutant_word):
  
  origin_sum, mutant_sum =  token_vec_sum(origin, word), token_vec_sum(mutant, mutant_word)
  return 1 - cosine(origin_sum, mutant_sum)
      
  
def mutant_pairs(examples):
  pairs=[]
  inputs = [ex["en"] for ex in examples]
  for i, input in enumerate(inputs):
    input_list = input.split()
    tokens = examples[i]['top_tokens']
    #print(tokens)
    token_idx = find_tokenidx(tokens[0], input_list)
    #print(input_list[token_idx])
    word = input_list[token_idx]

    input_list[token_idx] = "[MASK]"
    candidate = unmasker(" ".join(input_list))

    mutants = []
    for c in candidate[:3]:
      mutant = input_list.copy()
      mutant[token_idx] = c["token_str"]
      
      mutant_sent = " ".join(mutant)
      # if similarity(input, mutant_sent, word, c["token_str"]) >=0.9:
      mutants.append(" ".join(mutant))
    pairs.append({'en': input, 'mutants': mutants})
  return pairs

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
mutant_pairs(result)

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer('bert-base-nli-mean-tokens')
def sentence_sim(text1, text2):
    origin = model.encode(text1)
    mutant = model.encode(text2)
    return cosine_similarity(
    [origin],
    [mutant])[0][0]

In [7]:
from transformers import BertConfig, BertTokenizer, BertModel, RobertaTokenizer, RobertaModel, BertForMaskedLM
import os
import torch
os.environ['TRANSFORMERS_CACHE'] = '/data2/hanyings/.cache'
def bertInit():
    #config = Ber
    berttokenizer = BertTokenizer.from_pretrained('bert-large-cased')
    bertmodel = BertForMaskedLM.from_pretrained("bert-large-cased")#'/data/szy/bertlarge')
    bertori = BertModel.from_pretrained("bert-large-cased")#'/data/szy/bertlarge')
    #berttokenizer = RobertaTokenizer.from_pretrained('bert-large-uncased')
    #bertmodel = RoBertaForMaskedLM.from_pretrained('/data/szy/bertlarge')
    bertmodel.eval().cuda().to(torch.device("cuda:0"))
    bertori.eval().cuda()#.to(torch.device("cuda:1"))
    
    return bertmodel, berttokenizer, bertori

In [ ]:
tensor = torch.tensor([[101, 103, 24563, 4102, 1106, 1321, 170, 12020, 1107, 4507, 119, 1188, 1110, 1103, 1148, 1159, 1115, 170, 1244, 2325, 5748, 3907, 4102, 1106, 1301, 12020, 1107, 4507, 1170, 4811, 23300, 117, 2452, 1106, 1103, 1244, 2325, 3054, 5732, 12627, 119, 102]]).cuda()
F.softmax(bertmodel(tensor)[0], dim=-1).data

NameError: name 'F' is not defined

In [20]:
import string
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from copy import deepcopy
import time

K_Number = 100
Max_Mutants = 5

ft = time.time()

lcache = []
def BertM(bert, berttoken, inpori, bertori, mutate_idx):
    #global lcache
    # for k in lcache:
    #     if inpori == k[0]:
    #         return k[1], k[2]
    sentence = inpori

    tokens = berttoken.tokenize(sentence)
    batchsize = 1000 // len(tokens)

    gen = []
    ltokens = ["[CLS]"] + tokens + ["[SEP]"]

    try:
        encoding = [berttoken.convert_tokens_to_ids(ltokens[0:mutate_idx] + ["[MASK]"] + ltokens[mutate_idx + 1:])]#.cuda()
    except:
        return " ".join(tokens), gen
    p = []
    #print("encoding :", len(encoding))
    for i in range(0, len(encoding)):
        tensor = torch.tensor(encoding[i: min(len(encoding), i + batchsize)]).cuda()
        #print("tensor size", tensor.size())
        pre = F.softmax(bert(tensor)[0], dim=-1).data.cpu() # logits of the bertmodel, (1,42,28996)
        p.append(pre)

    pre = torch.cat(p, 0) # concate on 0 dimension 
    #print("pre size", pre.size()) #(40, 42, 28996), (token, layer, dimension in bert)

    tarl = [[tokens, -1]]


    # delete for loop 
    topk = torch.topk(pre[0][mutate_idx +1], K_Number)#.tolist()
    
    value = topk[0].numpy() #top values
    topk = topk[1].numpy().tolist() # index of the top values
    
    #print (topk)
    topkTokens = berttoken.convert_ids_to_tokens(topk)
    # tarl = []
    #print(topkTokens)
    for index in range(len(topkTokens)):
        #print(topkTokens[index], value[index])
        if value[index] < 0.00001:
            break
        tt = topkTokens[index]
        #print (tt)
        if tt in string.punctuation:
            continue
        if tt.strip() == tokens[mutate_idx].strip():
            continue
        l = deepcopy(tokens)
        l[mutate_idx] = tt
        tarl.append([l, mutate_idx, value[index]])
    #print("tarl:", tarl)
        
    if len(tarl) == 0:
        return " ".join(tokens), gen
    
    lDB = []
 
    for i in range(0, len(tarl), batchsize):
        lDB.append(bertori(torch.tensor([berttoken.convert_tokens_to_ids(["[CLS]"] + l[0] + ["[SEP]"]) for l in tarl[i: min(i + batchsize, len(tarl))]]).cuda())[0].data.cpu().numpy())
    lDB = np.concatenate(lDB, axis=0)
            

    lDA = lDB[0]
    assert len(lDB) == len(tarl)
    tarl = tarl[1:]
    lDB = lDB[1:]
    for t in range(len(lDB)):
        DB = lDB[t][tarl[t][1]]
        DA = lDA[tarl[t][1]]

        cossim = np.sum(DA * DB) / (np.sqrt(np.sum(DA * DA)) * np.sqrt(np.sum(DB * DB)))

        if cossim < 0.85:
            continue

        sen = " ".join(tarl[t][0])# + "\t!@#$%^& " + str(math.exp(value[index]))#.replace(" ##", "")
        
          
        gen.append([cossim, sen])
    # if len(lcache) > 4:
    #     lcache = lcache[1:]    

    # lcache.append([inpori, " ".join(tokens), gen])
    gen.sort(key=lambda x: x[0], reverse=True)
    return " ".join(tokens), gen#.replace(" ##", ""), gen


In [21]:
bertmodel, berttoken, bertori = bertInit()

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.

In [15]:
tar, gen = BertM(bertmodel, berttoken, "Teresa Mei chose to take a vacation in Switzerland . This is the first time that a United Kingdom prime minister chose to go vacation in Switzerland after Margaret Thatcher , according to the United Kingdom newspaper Daily Telegraph .",  bertori, 18)

In [17]:
tar

'Teresa Mei chose to take a vacation in Switzerland . This is the first time that a United Kingdom prime minister chose to go vacation in Switzerland after Margaret Thatcher , according to the United Kingdom newspaper Daily Telegraph .'

In [16]:
gen

[[0.98236746,
  'Teresa Mei chose to take a vacation in Switzerland . This is the first time that a United kingdom prime minister chose to go vacation in Switzerland after Margaret Thatcher , according to the United Kingdom newspaper Daily Telegraph .'],
 [0.97620964,
  'Teresa Mei chose to take a vacation in Switzerland . This is the first time that a United Kingdoms prime minister chose to go vacation in Switzerland after Margaret Thatcher , according to the United Kingdom newspaper Daily Telegraph .'],
 [0.9493239,
  'Teresa Mei chose to take a vacation in Switzerland . This is the first time that a United Britain prime minister chose to go vacation in Switzerland after Margaret Thatcher , according to the United Kingdom newspaper Daily Telegraph .'],
 [0.8952423,
  'Teresa Mei chose to take a vacation in Switzerland . This is the first time that a United UK prime minister chose to go vacation in Switzerland after Margaret Thatcher , according to the United Kingdom newspaper Daily T

In [47]:
def align_index(bertmodel, berttoken, line, bertori, token_index, top_tokens):
    result = []
    tokens = berttoken.tokenize(line)
    # print(tokens)
    tar=""
    for i in range(len(token_index)):
        idx = max(token_index[i]-1, 0)
        try: 
            tokens[idx].lower() == top_tokens[i].lower()
        except:
            print(idx)
            print(token_index)
            print(tokens)
            print(top_tokens[i])
        if tokens[idx].lower() == top_tokens[i].lower():
            tar, gen = BertM(bertmodel, berttoken, line, bertori, idx)
            # print(tokens[idx])
            # print(gen)
            result.append(gen[0])
        else:
            idx = max(token_index[i]-3, 0)
            while idx<min(token_index[i]+4, len(tokens)):
                if tokens[idx].lower() == top_tokens[i].lower():
                    tar, gen = BertM(bertmodel, berttoken, line, bertori, idx)
                    result.append(gen[0])
                    break
                idx +=1
    return tar, result

In [43]:
line="28-Year-Old Chef Found Dead at San Francisco Mall"
token_index=[
            8,
            10,
            7,
            11,
            6
        ]
top_tokens=[
            "Dead",
            "San",
            "Found",
            "Francisco",
            "Chef"
        ]
align_index(bertmodel, berttoken, line, bertori, token_index, top_tokens)

('28 - Year - Old Chef Found Dead at San Francisco Mall',
 [[0.968499, '28 - Year - Old Chef Found Missing at San Francisco Mall'],
  [0.9966399, '28 - Year - Old Chef Found Dead at New Francisco Mall'],
  [0.9750549, '28 - Year - Old Chef Shot Dead at San Francisco Mall'],
  [0.9335128, '28 - Year - Old Chef Found Dead at San Diego Mall'],
  [0.96899986, '28 - Year - Old Assistant Found Dead at San Francisco Mall']])

In [49]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [46]:
import json
# with open(sys.argv[1]) as f:
with open("../CAT/NewThres/TestGenerator-NMT/en_token.json") as f:
    sent_data = json.load(f)
f = open("../CAT/NewThres/TestGenerator-NMT/f_en_mu_grad.txt", "w")
fline = open("../CAT/NewThres/TestGenerator-NMT/f_en_mu_grad.index", "w")
bertmodel, berttoken, bertori = bertInit()

for i in tqdm(range(len(sent_data))):
    line = sent_data[i]["en"]
    index= sent_data[i]["token_index"]
    top_tokens = sent_data[i]["top_tokens"]
    #tag = nlp.pos_tag(line)

    tar, gen = align_index(bertmodel, berttoken, line, bertori, index, top_tokens)
    gen = sorted(gen)[::-1]
    count = 0
    for sen in gen:
        f.write(tar.strip() + "\n")
        f.write(sen[1].strip() + "\n")
        fline.write(str(i) + "\n")
        count += 1
        if count >= Max_Mutants:
            break
f.close()
fline.close()
print (time.time() - ft)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.

IndexError: list index out of range

In [26]:
import string
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from copy import deepcopy
import time

K_Number = 100
Max_Mutants = 5

ft = time.time()

lcache = []
def BertM2 (bert, berttoken, inpori, bertori):
    global lcache
    # for k in lcache:
    #     if inpori == k[0]:
    #         return k[1], k[2]
    sentence = inpori

    tokens = berttoken.tokenize(sentence)
    batchsize = 1000 // len(tokens)

    gen = []
    ltokens = ["[CLS]"] + tokens + ["[SEP]"]

    try:
        encoding = [berttoken.convert_tokens_to_ids(ltokens[0:i] + ["[MASK]"] + ltokens[i + 1:]) for i in range(1, len(ltokens) - 1)]#.cuda()
    except:
        return " ".join(tokens), gen
    p = []
    print("encoding :", len(encoding))
    for i in range(0, len(encoding), batchsize):
        tensor = torch.tensor(encoding[i: min(len(encoding), i + batchsize)]).cuda()
        print("tensor size", tensor.size())
        pre = F.softmax(bert(tensor)[0], dim=-1).data.cpu() # logits of the bertmodel, (1,42,28996)
        p.append(pre)

    pre = torch.cat(p, 0) # concate on 0 dimension 
    print("pre size", pre.size()) #(40, 42, 28996), (token, layer, dimension in bert)

    tarl = [[tokens, -1]]
    for i in range(len(tokens)):

        if tokens[i] in string.punctuation:
            continue
        topk = torch.topk(pre[i][i + 1], K_Number)#.tolist()
        
        value = topk[0].numpy() #top values
        topk = topk[1].numpy().tolist() # index of the top values
        
        #print (topk)
        topkTokens = berttoken.convert_ids_to_tokens(topk)
       # tarl = []
        #print(topkTokens)
        for index in range(len(topkTokens)):
            if value[index] < 0.05:
                break
            tt = topkTokens[index]
            #print (tt)
            if tt in string.punctuation:
                continue
            if tt.strip() == tokens[i].strip():
                continue
            l = deepcopy(tokens)
            l[i] = tt
            tarl.append([l, i, value[index]])
        #print("tarl:", tarl)
        
    if len(tarl) == 0:
        return " ".join(tokens), gen
        
    
    
    lDB = []
 
    for i in range(0, len(tarl), batchsize):
        lDB.append(bertori(torch.tensor([berttoken.convert_tokens_to_ids(["[CLS]"] + l[0] + ["[SEP]"]) for l in tarl[i: min(i + batchsize, len(tarl))]]).cuda())[0].data.cpu().numpy())
    lDB = np.concatenate(lDB, axis=0)
            

    lDA = lDB[0]
    assert len(lDB) == len(tarl)
    tarl = tarl[1:]
    lDB = lDB[1:]
    for t in range(len(lDB)):
        DB = lDB[t][tarl[t][1]]
        DA = lDA[tarl[t][1]]

        cossim = np.sum(DA * DB) / (np.sqrt(np.sum(DA * DA)) * np.sqrt(np.sum(DB * DB)))

        if cossim < 0.85:
            continue

        sen = " ".join(tarl[t][0])# + "\t!@#$%^& " + str(math.exp(value[index]))#.replace(" ##", "")
        
          
        gen.append([cossim, sen])
    if len(lcache) > 4:
        lcache = lcache[1:]    

    lcache.append([inpori, " ".join(tokens), gen])
    gen.sort(key=lambda x: x[0], inverse=True)
    return " ".join(tokens), gen#.replace(" ##", ""), gen


In [27]:
tar2, gen2 = BertM2(bertmodel, berttoken, "Teresa Mei chose to take a vacation in Switzerland . This is the first time that a United Kingdom prime minister chose to go vacation in Switzerland after Margaret Thatcher , according to the United Kingdom newspaper Daily Telegraph .",  bertori)
gen2

encoding : 40
tensor size torch.Size([25, 42])
tensor size torch.Size([15, 42])
pre size torch.Size([40, 42, 28996])


[[0.99286634,
  'Li Mei chose to take a vacation in Switzerland . This is the first time that a United Kingdom prime minister chose to go vacation in Switzerland after Margaret Thatcher , according to the United Kingdom newspaper Daily Telegraph .'],
 [0.98540044,
  'Chen Mei chose to take a vacation in Switzerland . This is the first time that a United Kingdom prime minister chose to go vacation in Switzerland after Margaret Thatcher , according to the United Kingdom newspaper Daily Telegraph .'],
 [0.9864119,
  'Teresa also chose to take a vacation in Switzerland . This is the first time that a United Kingdom prime minister chose to go vacation in Switzerland after Margaret Thatcher , according to the United Kingdom newspaper Daily Telegraph .'],
 [0.98795044,
  'Teresa May chose to take a vacation in Switzerland . This is the first time that a United Kingdom prime minister chose to go vacation in Switzerland after Margaret Thatcher , according to the United Kingdom newspaper Daily T